In [1]:
from Acesso import Login
from Query import Query
import pandas as pd

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

In [2]:
querys={
    
    'Equipamento':
    
    """
    
    SELECT *
    FROM netfeira.vw_venda_estatico ped
    WHERE ped.[Tipo de Pedido] LIKE '%COMODATO%'    
    
    """,
    
    'Cliente':
    
    """
    
    SELECT * 
    FROM netfeira.vw_cliente cli
    INNER JOIN netfeira.vw_vendedor vend ON cli.Principal=vend.[ID Vendedor]
    INNER JOIN netfeira.vw_supervisor sup ON vend.[ID Equipe]=sup.[ID Equipe]
    
    """
}

In [3]:
df=sql.CriarTabela(kwargs=querys)

In [6]:
df['Temp']=df['Equipamento'].groupby(['ID Cliente'],as_index=False).agg({'Total Venda':'sum'})

In [9]:
df['Temp']=df['Temp'].loc[df['Temp']['Total Venda']>0]

In [15]:
codigos=df['Temp']['ID Cliente'].unique().tolist()

In [16]:
df['Cliente'].columns

Index(['ID Cliente', 'CNPJ', 'Rede', 'CNPJ Base', 'CNPJ Caracter',
       'Razão Social', 'Nome Fantasia', 'Tipo de Cliente', 'Status do Cliente',
       'ID Segmento', 'Matriz', 'Crédito', 'Data de Cadastro',
       'Primeira Compra', 'Última Compra', 'Dias Compra', 'Tabela',
       'Condição de Pagto', 'Prazo Pagto', 'Pagamento', 'CEP', 'Endereço',
       'Bairro', 'Município', 'Numero', 'Complemento', 'Região', 'UF', 'DDD',
       'Contato', 'Limite de Crédito', 'Principal', 'E-mail Cliente',
       'Tributação do Cliente', 'ID Rota', 'Latitude', 'Longitude',
       'Desc Financeiro', 'ID Vendedor', 'Vendedor', 'Nome Resumido',
       'ID Equipe', 'E-mail', 'Categoria', 'Data de Cadastro',
       'Status do Vendedor', 'DDD', 'Telefone', 'ID Equipe', 'Equipe',
       'ID Sup', 'Supervisor', 'Email Sup', 'DDD Sup', 'Telefone Sup',
       'ID Gerente', 'Gerente', 'Email Gerente', 'DDD Gerente',
       'Telefone Gerente'],
      dtype='object')

In [23]:
df['Temp']=df['Cliente'][['ID Cliente','Razão Social', 'Nome Fantasia','Matriz','CEP', 'Endereço',
       'Bairro', 'Município', 'Numero', 'Complemento', 'Região', 'UF', 'DDD',
       'Contato','ID Vendedor', 'Vendedor','Equipe','Supervisor']].loc[df['Cliente']['ID Cliente'].isin(codigos)]

In [26]:
df['Temp']['Equipamentos']=df['Temp']['ID Cliente'].apply(lambda info: len(df['Equipamento']['SKU'].loc[df['Equipamento']['ID Cliente']==info].unique().tolist()))

In [29]:
df['Temp'].sort_values('Equipamentos',ascending=False,inplace=True)

In [32]:
df['Temp']=df['Temp'].loc[df['Temp']['Supervisor'].str.contains('ULISSES')]

In [37]:
df['Temp'].to_excel('Equipamentos.xlsx',index=False,encoding='UTF-8')